In [46]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
from sklearn import tree
import graphviz
from sklearn.model_selection import cross_val_score

In [2]:
#laod the dataset provided
salary_dataset  = pd.read_csv('/home/abhi/Desktop/adult.csv')

# describe the dataset 
salary_dataset.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [3]:
 
salary_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
salary_dataset.isnull().sum()


age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [5]:
df = pd.DataFrame(salary_dataset)
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [6]:
#replacing some special character columns names with proper names 
df.rename(columns={'capital-gain': 'capital gain', 'capital-loss': 'capital loss', 'native-country': 'country','hours-per-week': 'hours per week','marital-status': 'marital'}, inplace=True)

In [7]:

df.columns


Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital',
       'occupation', 'relationship', 'race', 'gender', 'capital gain',
       'capital loss', 'hours per week', 'country', 'income'],
      dtype='object')

In [8]:
#Finding the special characters in the data frame 
df.isin(['?']).sum(axis=0)

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital               0
occupation         2809
relationship          0
race                  0
gender                0
capital gain          0
capital loss          0
hours per week        0
country             857
income                0
dtype: int64

In [9]:
# Create a new income_bi column
df['income_bi'] = df.apply(lambda row: 1 if '>50K'in row['income'] else 0, axis=1)
# Remove redundant columns


In [10]:
# the code will replace the special character to nan and then drop the columns 
df['country'] = df['country'].replace('?',np.nan)
df['workclass'] = df['workclass'].replace('?',np.nan)
df['occupation'] = df['occupation'].replace('?',np.nan)

In [11]:
 
df.dropna(how='any',inplace=True)

In [12]:
 
df.isin(['?']).sum(axis=0)

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital            0
occupation         0
relationship       0
race               0
gender             0
capital gain       0
capital loss       0
hours per week     0
country            0
income             0
income_bi          0
dtype: int64

In [13]:
#running a loop for value_counts of each column to find out unique values. 
for c in df.columns:
    print ("---- %s ---" % c)
    print (df[c].value_counts())

---- age ---
36    1283
33    1279
31    1274
35    1272
23    1241
      ... 
85       5
88       5
86       1
87       1
89       1
Name: age, Length: 74, dtype: int64
---- workclass ---
Private             33307
Self-emp-not-inc     3796
Local-gov            3100
State-gov            1946
Self-emp-inc         1646
Federal-gov          1406
Without-pay            21
Name: workclass, dtype: int64
---- fnlwgt ---
203488    21
125892    18
120277    18
113364    17
126569    17
          ..
88440      1
176517     1
194956     1
201105     1
208174     1
Name: fnlwgt, Length: 26741, dtype: int64
---- education ---
HS-grad         14783
Some-college     9899
Bachelors        7570
Masters          2514
Assoc-voc        1959
11th             1619
Assoc-acdm       1507
10th             1223
7th-8th           823
Prof-school       785
9th               676
12th              577
Doctorate         544
5th-6th           449
1st-4th           222
Preschool          72
Name: education, dtype: int

In [14]:
 
df.workclass.value_counts()

Private             33307
Self-emp-not-inc     3796
Local-gov            3100
State-gov            1946
Self-emp-inc         1646
Federal-gov          1406
Without-pay            21
Name: workclass, dtype: int64

In [15]:
#dropping un-used data from the dataset 
df.drop(['educational-num','age', 'hours per week', 'fnlwgt', 'capital gain','capital loss', 'country'], axis=1, inplace=True)

In [16]:
# Let's see how many unique categories we have in this property
income = set(df['income'])
print(income)

{'<=50K', '>50K'}


In [17]:

df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)

In [18]:
 
df.head()

,workclass,education,marital,occupation,relationship,race,gender,income,income_bi
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,1,1
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,1,1
5,Private,10th,Never-married,Other-service,Not-in-family,White,Male,0,0


In [19]:
# Let's see how many unique categories we have in this gender property
gender = set(df['gender'])
print(gender)

{'Male', 'Female'}


In [20]:
#Mapping the values to numerical values 
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1}).astype(int)

In [21]:
race = set(df['race'])
print(race)

{'Black', 'Amer-Indian-Eskimo', 'White', 'Other', 'Asian-Pac-Islander'}


In [22]:
#Mapping the values to numerical values 
df['race'] = df['race'].map({'Black': 0, 'Asian-Pac-Islander': 1, 'Other': 2, 'White': 3, 
                                             'Amer-Indian-Eskimo': 4}).astype(int)

In [23]:
Marital = set(df['marital'])
print(Marital)

{'Married-civ-spouse', 'Married-AF-spouse', 'Widowed', 'Divorced', 'Never-married', 'Separated', 'Married-spouse-absent'}


In [24]:
df['marital'] = df['marital'].map({'Married-spouse-absent': 0, 'Widowed': 1, 
                                                             'Married-civ-spouse': 2, 'Separated': 3, 'Divorced': 4, 
                                                             'Never-married': 5, 'Married-AF-spouse': 6}).astype(int)

In [25]:
# How many unique Workclass we got here?
emp = set(df['workclass'])
print(emp)

{'Without-pay', 'Self-emp-inc', 'Local-gov', 'Private', 'Federal-gov', 'Self-emp-not-inc', 'State-gov'}


In [26]:
#Mapping the values to numerical values
df['workclass'] = df['workclass'].map({'Self-emp-inc': 0, 'State-gov': 1, 
                                                             'Federal-gov': 2, 'Without-pay': 3, 'Local-gov': 4, 
                                                             'Private': 5, 'Self-emp-not-inc': 6}).astype(int)

In [27]:
# How many unique Education we got here?
ed = set(df['education'])
print(ed)


{'11th', '5th-6th', 'Preschool', '1st-4th', 'Assoc-acdm', '7th-8th', 'Doctorate', 'Assoc-voc', 'Masters', 'HS-grad', 'Bachelors', '10th', '9th', '12th', 'Prof-school', 'Some-college'}


In [28]:
#Mapping the values to numerical values
df['education'] = df['education'].map({'Some-college': 0, 'Preschool': 1, 
                                                        '5th-6th': 2, 'HS-grad': 3, 'Masters': 4, 
                                                        '12th': 5, '7th-8th': 6, 'Prof-school': 7,
                                                        '1st-4th': 8, 'Assoc-acdm': 9,
                                                        'Doctorate': 10, '11th': 11,
                                                        'Bachelors': 12, '10th': 13,
                                                        'Assoc-voc': 14,
                                                        '9th': 15}).astype(int)

In [29]:

occupation = set(df['occupation'])
print(occupation)

{'Protective-serv', 'Sales', 'Machine-op-inspct', 'Priv-house-serv', 'Other-service', 'Prof-specialty', 'Farming-fishing', 'Handlers-cleaners', 'Transport-moving', 'Tech-support', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Adm-clerical'}


In [30]:
# Now we classify them as numbers instead of their names.
df['occupation'] = df['occupation'].map({ 'Farming-fishing': 1, 'Tech-support': 2, 
                                          'Adm-clerical': 3, 'Handlers-cleaners': 4, 
                                         'Prof-specialty': 5,'Machine-op-inspct': 6, 
                                         'Exec-managerial': 7, 
                                         'Priv-house-serv': 8,
                                         'Craft-repair': 9, 
                                         'Sales': 10, 
                                         'Transport-moving': 11, 
                                         'Armed-Forces': 12, 
                                         'Other-service': 13,  
                                         'Protective-serv': 14}).astype(int)

In [31]:
# How many unique Relationship we got here?
relationship = set(df['relationship'])
print(relationship)
#Mapping the values to numerical values
df['relationship'] = df['relationship'].map({'Not-in-family': 0, 'Wife': 1, 
                                                             'Other-relative': 2, 
                                                             'Unmarried': 3, 
                                                             'Husband': 4, 
                                                             'Own-child': 5}).astype(int)

{'Wife', 'Unmarried', 'Husband', 'Own-child', 'Other-relative', 'Not-in-family'}


In [32]:
#displaying the cleaned data to see if the map as worked
df.head(10)

,workclass,education,marital,occupation,relationship,race,gender,income,income_bi
0,5,11,5,6,5,0,0,0,0
1,5,3,2,1,4,3,0,0,0
2,4,9,2,14,4,3,0,1,1
3,5,0,2,6,4,0,0,1,1
5,5,13,5,13,0,3,0,0,0
7,6,7,2,5,4,3,0,1,1
8,5,0,5,13,3,3,1,0,0
9,5,6,2,9,4,3,0,0,0
10,5,3,2,6,4,3,0,1,1
11,2,12,2,3,4,3,0,0,0


In [33]:

entropy_node = 0  
values = df.workclass.unique() 
for value in values:
    fraction = df.workclass.value_counts()[value]/len(df.workclass)  
    entropy_node += -fraction*np.log2(fraction)

In [34]:
entropy_node

1.42017117756034

In [35]:
def ent(df,attribute):
    target_variables = df.workclass.unique()  
    variables = df[attribute].unique()    


    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.workclass ==target_variable])
            den = len(df[attribute][df[attribute]==variable]) 
            fraction = num/(den+eps)  #pi
            entropy_each_feature += -fraction*log(fraction+eps) 
        fraction2 = den/len(df)
        entropy_attribute += -fraction2*entropy_each_feature   

    return(abs(entropy_attribute))

In [36]:
a_entropy = {k:ent(df,k) for k in df.keys()[:-1]}
a_entropy

{'workclass': 3.203426503814917e-16,
 'education': 1.3774490848754897,
 'marital': 1.3931031592333591,
 'occupation': 1.252158700371167,
 'relationship': 1.391122017102175,
 'race': 1.4091975637172625,
 'gender': 1.4034143814952238,
 'income': 1.4031382083435704}

In [37]:
def ig(e_dataset,e_attr):
    return(e_dataset-e_attr)

In [38]:
#entropy_node = entropy of dataset
#a_entropy[k] = entropy of k(th) attr
IG = {k:ig(entropy_node,a_entropy[k]) for k in a_entropy}
IG

{'workclass': 1.4201711775603398,
 'education': 0.04272209268485039,
 'marital': 0.027068018326980914,
 'occupation': 0.16801247718917312,
 'relationship': 0.02904916045816508,
 'race': 0.010973613843077601,
 'gender': 0.016756796065116264,
 'income': 0.017032969216769667}

In [39]:
#t=buildTree(df)

In [40]:
#import pprint

In [41]:
#pprint.pprint(t)

In [42]:
# shuffle rows
df = df.sample(frac=1)
# split training and testing data
d_train = df[:25000]
d_test = df[25000:]
d_train_att = d_train.drop(['income_bi'], axis=1)
d_train_gt50 = d_train['income_bi']
d_test_att = d_test.drop(['income_bi'], axis=1)
d_test_gt50 = d_test['income_bi']
d_att = df.drop(['income_bi'], axis=1)
d_gt50 = df['income_bi']
# number of income > 50K in whole dataset:
print("Income >50K: %d out of %d (%.2f%%)" % (np.sum(d_gt50), len(d_gt50), 100*float(np.sum(d_gt50)) / len(d_gt50)))
# Income >50K: 7508 out of 30162 (24.89%)

Income >50K: 11208 out of 45222 (24.78%)


In [47]:
# Fit a decision tree
t = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7)
t = t.fit(d_train_att, d_train_gt50)

In [49]:
t.score(d_test_att, d_test_gt50)
# 0.820030995738086

1.0

In [50]:
scores = cross_val_score(t, d_att, d_gt50, cv=5)
# Show avarage score and +/- two standard deviations away (covering 95% or scores)
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std()*2))
# Accuracy: 0.83 (+/- 0.00)

Accuracy: 1.00 (+/- 0.00)


In [51]:
# Create a sample csv for prediction
df.iloc[[0]].to_csv('prediction.csv', sep=',', encoding='utf-8', index=False)

In [52]:
# Prepare user profile
sample_df = pd.read_csv('prediction.csv', sep=',')
sample_df = sample_df.drop(['income_bi'], axis=1)
# Start predicting 
predict_value = sample_df.iloc[0]
y_predict = t.predict([predict_value.tolist()])
y_predict[0] #0

0

In [53]:
for max_depth in range(1, 20):
    t = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    scores = cross_val_score(t, d_att, d_gt50, cv=5)
    print("Max depth: %d, Accuracy: %0.2f (+/- %0.2f)" % (max_depth, scores.mean(), scores.std()*2))

Max depth: 1, Accuracy: 1.00 (+/- 0.00)
Max depth: 2, Accuracy: 1.00 (+/- 0.00)
Max depth: 3, Accuracy: 1.00 (+/- 0.00)
Max depth: 4, Accuracy: 1.00 (+/- 0.00)
Max depth: 5, Accuracy: 1.00 (+/- 0.00)
Max depth: 6, Accuracy: 1.00 (+/- 0.00)
Max depth: 7, Accuracy: 1.00 (+/- 0.00)
Max depth: 8, Accuracy: 1.00 (+/- 0.00)
Max depth: 9, Accuracy: 1.00 (+/- 0.00)
Max depth: 10, Accuracy: 1.00 (+/- 0.00)
Max depth: 11, Accuracy: 1.00 (+/- 0.00)
Max depth: 12, Accuracy: 1.00 (+/- 0.00)
Max depth: 13, Accuracy: 1.00 (+/- 0.00)
Max depth: 14, Accuracy: 1.00 (+/- 0.00)
Max depth: 15, Accuracy: 1.00 (+/- 0.00)
Max depth: 16, Accuracy: 1.00 (+/- 0.00)
Max depth: 17, Accuracy: 1.00 (+/- 0.00)
Max depth: 18, Accuracy: 1.00 (+/- 0.00)
Max depth: 19, Accuracy: 1.00 (+/- 0.00)


In [69]:
#Function to calculate final Entropy 

def entropy(probs):  

    import math

    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#Function to calculate Probabilities of positive and negative examples 

def entropy_of_list(a_list):

    from collections import Counter

    cnt = Counter(x for x in a_list) #Count the positive and negative ex

    num_instances = len(a_list)

    #Calculate the probabilities that we required for our entropy formula 

    probs = [x / num_instances for x in cnt.values()] 

    #Calling entropy function for final entropy 

    return entropy(probs)

total_entropy = entropy_of_list(df['workclass'])

print("\n Total Entropy of PlayTennis Data Set:",total_entropy)



 Total Entropy of PlayTennis Data Set: 1.42017117756034
<function entropy_of_list at 0x7f07822ea1f0>


In [70]:

#Defining Information Gain Function 

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):

    print("Information Gain Calculation of ",split_attribute_name)

    print("target_attribute_name",target_attribute_name)

#Grouping features of Current Attribute

    df_split = df.groupby(split_attribute_name)

    for name,group in df_split:

        print("Name: ",name)

        print("Group: ",group)

    nobs = len(df.index) * 1.0

    print("NOBS",nobs)

#Calculating Entropy of the Attribute and probability part of formula 

    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]

    print("df_agg_ent",df_agg_ent)

# Calculate Information Gain

    avg_info = sum( df_agg_ent['Entropy'] * df_agg_ent['Prob1'] )

    old_entropy = entropy_of_list(df[target_attribute_name])

    return old_entropy - avg_info

print('Info-gain for Outlook is :'+str(information_gain(df, 'income_bi', 'workclass')),"\n")

Information Gain Calculation of  income_bi
target_attribute_name workclass
Name:  0
Group:         workclass  education  marital  occupation  relationship  race  gender  \
34632          5          0        5           3             0     3       0   
30879          5          3        4           3             3     3       1   
10786          5          0        4           8             0     3       1   
16555          5          3        4           5             5     3       1   
18649          5          3        5           9             5     3       0   
...          ...        ...      ...         ...           ...   ...     ...   
48657          5          3        4           9             0     1       1   
44975          0          3        2           7             4     3       0   
35577          6          3        2           7             4     3       0   
1019           5          3        2           2             4     0       0   
8849           5          3 

KeyError: 'entropy'